# Analysis: Agent social problem solving ability

How will do current pretrained LLMs do at social problem solving?

## Run experiment

In [ ]:
from pathlib import Path

from src.env import Env
from src.state import EnvState, AgentState
from src.actionmodel import RandomActionModel
from src.experiment import Experiment

In [ ]:
n_agents = 4
n_rounds = 10
energy_usage_rate = 100
starting_agent_energy = 300
total_energy = n_agents * n_rounds * energy_usage_rate


def random_strategy_env():
    Env.register_model(model_name="random", model=RandomActionModel(trxn_amount=100))
                       
    agents = [
        AgentState(id=f"agent_{i}", energy=starting_agent_energy, model="random")
        for i in range(n_agents)
    ]
    env = Env(
        EnvState(
            available_energy=(total_energy - starting_agent_energy * n_agents),
            energy_usage_rate=energy_usage_rate,
            agents=agents,
            n_rounds=n_rounds,
            receiver_agent=agents[0].id,
        )
    )
    return env


def always_skip_strategy_env():
    agents = [
        AgentState(id=f"agent_{i}", energy=starting_agent_energy, model="always-skip")
        for i in range(n_agents)
    ]
    env = Env(
        EnvState(
            available_energy=(total_energy - starting_agent_energy * n_agents),
            energy_usage_rate=energy_usage_rate,
            agents=agents,
            n_rounds=n_rounds,
            receiver_agent=agents[0].id,
        )
    )
    return env

In [ ]:
experiment = Experiment(
    name="random_vs_always_skip",
    n_replications=10,
    variants={
        "random_strategy": random_strategy_env,
        "always_skip_strategy": always_skip_strategy_env,
    }
)

experiment.run(log_dir=Path.cwd() / "data/")

## Analyze experiments

Questions:
- How does performance at this social game compare across action models?
- How does the distribution of energy (across agents over time) compare across action models?
- How does altruistic vs self-serving behaviour compare across action models?
    - What is the rate of giving?
    - What is the rate of taking?
    - What is the ratio of current energy to energy given?
    - What is the ratio of current energy to energy taken?

### Read experiment data

Tables:
- ExperimentTrials:
    - Experiment name
    - Variant
    - Trial
    - Score
    - Max score

- ActionLogs:
    - Experiment name
    - Variant
    - Trial
    - Agent
    - Round
    - Action